In [3]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Input
from tensorflow.keras import layers, Model, regularizers
import tensorflow as tf


base_model = EfficientNetB0(include_top=False, weights='imagenet')

# Unfreeze dari layer ke-
for layer in base_model.layers[:163]:
    layer.trainable = False
for layer in base_model.layers[163:]:
    layer.trainable = True

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0,2)(x)
outputs = Dense(32, activation='softmax')(x)


model = Model(inputs=base_model.input, outputs=outputs)
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print(len(model.layers))

# model.summary()


for i, layer in enumerate(model.layers):
    print(f"{i}: {layer.name}")

layer_index = 163
print(f"Layer ke-{layer_index}: {model.layers[layer_index].name}")

241
0: input_layer_2
1: rescaling_4
2: normalization_2
3: rescaling_5
4: stem_conv_pad
5: stem_conv
6: stem_bn
7: stem_activation
8: block1a_dwconv
9: block1a_bn
10: block1a_activation
11: block1a_se_squeeze
12: block1a_se_reshape
13: block1a_se_reduce
14: block1a_se_expand
15: block1a_se_excite
16: block1a_project_conv
17: block1a_project_bn
18: block2a_expand_conv
19: block2a_expand_bn
20: block2a_expand_activation
21: block2a_dwconv_pad
22: block2a_dwconv
23: block2a_bn
24: block2a_activation
25: block2a_se_squeeze
26: block2a_se_reshape
27: block2a_se_reduce
28: block2a_se_expand
29: block2a_se_excite
30: block2a_project_conv
31: block2a_project_bn
32: block2b_expand_conv
33: block2b_expand_bn
34: block2b_expand_activation
35: block2b_dwconv
36: block2b_bn
37: block2b_activation
38: block2b_se_squeeze
39: block2b_se_reshape
40: block2b_se_reduce
41: block2b_se_expand
42: block2b_se_excite
43: block2b_project_conv
44: block2b_project_bn
45: block2b_drop
46: block2b_add
47: block3a_e

In [5]:
def ctc_lambda_func(args):
    y_pred, labels, input_length, label_length = args
    return tf.keras.backend.ctc_batch_cost(labels, y_pred, input_length, label_length)


input_shape=(224, 224, 3)
vocab_size=51

input_img = layers.Input(shape=input_shape, name='image_ctc')
base_model = EfficientNetB0(include_top=False, input_tensor=input_img, weights='imagenet')

FINE_TUNE = True
if not FINE_TUNE:
    base_model.trainable = False  # Bekukan semua layer
else:
    # Opsi 1: Fine-tune sebagian dari layer (misal mulai dari layer ke-200)
    fine_tune_at = 163
    for layer in base_model.layers[:fine_tune_at]:
        layer.trainable = False
    for layer in base_model.layers[fine_tune_at:]:
        layer.trainable = True

x = base_model.output
x = layers.Conv2D(256, (3, 3), padding='same', activation='relu')(x)
x = layers.Reshape((x.shape[1], x.shape[2] * x.shape[3]))(x)
x = layers.Bidirectional(layers.LSTM(128, return_sequences=True))(x)
x = layers.Bidirectional(layers.LSTM(128, return_sequences=True))(x)
x = layers.Dropout(0.2)(x)
logits = layers.Dense(vocab_size, activation='softmax', name='logits')(x)

labels = layers.Input(name='labels', shape=(None,), dtype='int32')
input_length = layers.Input(name='input_length', shape=(1,), dtype='int32')
label_length = layers.Input(name='label_length', shape=(1,), dtype='int32')
loss_out = layers.Lambda(ctc_lambda_func, name='ctc')([logits, labels, input_length, label_length])

model = Model(inputs=[input_img, labels, input_length, label_length], outputs=loss_out)

print(len(model.layers))

model.summary()


for i, layer in enumerate(model.layers):
    print(f"{i}: {layer.name}")


248


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ image_ctc           │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_2         │ (None, 224, 224,  │          0 │ image_ctc[0][0]   │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization_1     │ (None, 224, 224,  │          7 │ rescaling_2[0][0] │
│ (Normalization)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_3         │ (None, 224, 224,  │          0 │ normalization_1[… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv_pad       │ (None, 225, 225,  │          0 │ rescaling_3[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 112, 112,  │        864 │ stem_conv_pad[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (None, 112, 112,  │        128 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation     │ (None, 112, 112,  │          0 │ stem_bn[0][0]     │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_dwconv      │ (None, 112, 112,  │        288 │ stem_activation[… │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_bn          │ (None, 112, 112,  │        128 │ block1a_dwconv[0… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_activation  │ (None, 112, 112,  │          0 │ block1a_bn[0][0]  │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_squeeze  │ (None, 32)        │          0 │ block1a_activati… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reshape  │ (None, 1, 1, 32)  │          0 │ block1a_se_squee… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reduce   │ (None, 1, 1, 8)   │        264 │ block1a_se_resha… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_expand   │ (None, 1, 1, 32)  │        288 │ block1a_se_reduc… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_excite   │ (None, 112, 112,  │          0 │ block1a_activati… │
│ (Multiply)          │ 32)               │            │ block1a_se_expan… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_co… │ (None, 112, 112,  │        512 │ block1a_se_excit

 Total params: 9,373,398 (35.76 MB)

 Trainable params: 8,479,567 (32.35 MB)

 Non-trainable params: 893,831 (3.41 MB)

0: image_ctc
1: rescaling_2
2: normalization_1
3: rescaling_3
4: stem_conv_pad
5: stem_conv
6: stem_bn
7: stem_activation
8: block1a_dwconv
9: block1a_bn
10: block1a_activation
11: block1a_se_squeeze
12: block1a_se_reshape
13: block1a_se_reduce
14: block1a_se_expand
15: block1a_se_excite
16: block1a_project_conv
17: block1a_project_bn
18: block2a_expand_conv
19: block2a_expand_bn
20: block2a_expand_activation
21: block2a_dwconv_pad
22: block2a_dwconv
23: block2a_bn
24: block2a_activation
25: block2a_se_squeeze
26: block2a_se_reshape
27: block2a_se_reduce
28: block2a_se_expand
29: block2a_se_excite
30: block2a_project_conv
31: block2a_project_bn
32: block2b_expand_conv
33: block2b_expand_bn
34: block2b_expand_activation
35: block2b_dwconv
36: block2b_bn
37: block2b_activation
38: block2b_se_squeeze
39: block2b_se_reshape
40: block2b_se_reduce
41: block2b_se_expand
42: block2b_se_excite
43: block2b_project_conv
44: block2b_project_bn
45: block2b_drop
46: block2b_add
47: block3a_expand_co

In [ ]:
from tensorflow.keras.applications import EfficientNetB1
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Input
from tensorflow.keras import layers, Model, regularizers
import tensorflow as tf


base_model = EfficientNetB1(include_top=False, weights='imagenet')

# Unfreeze dari layer ke-
for layer in base_model.layers[:235]:
    layer.trainable = False
for layer in base_model.layers[235:]:
    layer.trainable = True

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0,2)(x)
outputs = Dense(32, activation='softmax')(x)


model = Model(inputs=base_model.input, outputs=outputs)
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print(len(model.layers))

# model.summary()


for i, layer in enumerate(model.layers):
    print(f"{i}: {layer.name}")

layer_index = 163
print(f"Layer ke-{layer_index}: {model.layers[layer_index].name}")

343
0: input_layer_1
1: rescaling_2
2: normalization_1
3: rescaling_3
4: stem_conv_pad
5: stem_conv
6: stem_bn
7: stem_activation
8: block1a_dwconv
9: block1a_bn
10: block1a_activation
11: block1a_se_squeeze
12: block1a_se_reshape
13: block1a_se_reduce
14: block1a_se_expand
15: block1a_se_excite
16: block1a_project_conv
17: block1a_project_bn
18: block1b_dwconv
19: block1b_bn
20: block1b_activation
21: block1b_se_squeeze
22: block1b_se_reshape
23: block1b_se_reduce
24: block1b_se_expand
25: block1b_se_excite
26: block1b_project_conv
27: block1b_project_bn
28: block1b_drop
29: block1b_add
30: block2a_expand_conv
31: block2a_expand_bn
32: block2a_expand_activation
33: block2a_dwconv_pad
34: block2a_dwconv
35: block2a_bn
36: block2a_activation
37: block2a_se_squeeze
38: block2a_se_reshape
39: block2a_se_reduce
40: block2a_se_expand
41: block2a_se_excite
42: block2a_project_conv
43: block2a_project_bn
44: block2b_expand_conv
45: block2b_expand_bn
46: block2b_expand_activation
47: block2b_d

In [1]:
from tensorflow.keras.applications import EfficientNetB2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Input
from tensorflow.keras import layers, Model, regularizers
import tensorflow as tf


base_model = EfficientNetB2(include_top=False, weights='imagenet')

# Unfreeze dari layer ke-
for layer in base_model.layers[:163]:
    layer.trainable = False
for layer in base_model.layers[163:]:
    layer.trainable = True

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0,2)(x)
outputs = Dense(32, activation='softmax')(x)


model = Model(inputs=base_model.input, outputs=outputs)
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print(len(model.layers))

# model.summary()

for i, layer in enumerate(model.layers):
    print(f"{i}: {layer.name}")

layer_index = 163
print(f"Layer ke-{layer_index}: {model.layers[layer_index].name}")

I0000 00:00:1754927744.860276 1501832 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9248 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060, pci bus id: 0000:2b:00.0, compute capability: 8.6


343
0: input_layer
1: rescaling
2: normalization
3: rescaling_1
4: stem_conv_pad
5: stem_conv
6: stem_bn
7: stem_activation
8: block1a_dwconv
9: block1a_bn
10: block1a_activation
11: block1a_se_squeeze
12: block1a_se_reshape
13: block1a_se_reduce
14: block1a_se_expand
15: block1a_se_excite
16: block1a_project_conv
17: block1a_project_bn
18: block1b_dwconv
19: block1b_bn
20: block1b_activation
21: block1b_se_squeeze
22: block1b_se_reshape
23: block1b_se_reduce
24: block1b_se_expand
25: block1b_se_excite
26: block1b_project_conv
27: block1b_project_bn
28: block1b_drop
29: block1b_add
30: block2a_expand_conv
31: block2a_expand_bn
32: block2a_expand_activation
33: block2a_dwconv_pad
34: block2a_dwconv
35: block2a_bn
36: block2a_activation
37: block2a_se_squeeze
38: block2a_se_reshape
39: block2a_se_reduce
40: block2a_se_expand
41: block2a_se_excite
42: block2a_project_conv
43: block2a_project_bn
44: block2b_expand_conv
45: block2b_expand_bn
46: block2b_expand_activation
47: block2b_dwconv
